**Pendulum environment - Stable baselines**

Vamos direto ao assunto:

In [1]:
!apt-get install -y xvfb x11-utils
!pip install gym[box2d]==0.17.* pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libxxf86dga1
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libxxf86dga1 x11-utils xvfb
0 upgraded, 3 newly installed, 0 to remove and 12 not upgraded.
Need to get 994 kB of archives.
After this operation, 2,982 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 x11-utils amd64 7.7+3build1 [196 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.11 [785 kB]
Fetched 994 kB in 1s (1,224 kB/s)
Selecting previously unselected package libxxf86dga1:amd64.
(Reading database ... 123934 fil

In [2]:
!pip install stable-baselines3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 170 kB 5.4 MB/s 
     |████████████████████████████████| 1.5 MB 50.6 MB/s 
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616823 sha256=905587adce2419dd0234c0b499f2e6a70b58d9143a16fc51960d53fa50607dd6
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff2771b34447d6d2f2148
Successfully built gym
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.0.0
    Uninstalling importlib-metadata-5.0.0:
      Successfully uninstalled importlib-metadata-5.0.0
  Attempting uninstall: gym
    Found existing installation: gym 0.17.3
    Uninstalling gym-0.17.3:
      Successfully uninstalled gym-0.17.3


In [3]:
import gym

Escolhendo o ambiente:

In [4]:
env = gym.make("Pendulum-v1")
env.seed(0)

[0]

Gerando estado aleatório:

Sample: [x, y, w]

*  x: posição_x da ponta; 
*  y: posição_y da ponta;
*  w: velocidade angular




In [5]:
print(env.observation_space.sample())

[-0.3702345 -0.6196617 -3.9916358]


Código para renderizar o ambiente:

In [6]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import display_html
from tqdm.notebook import tqdm

_frames = []

def salva_frame(env):
    _frames.append({
        'img': env.render(mode='rgb_array'),
    })
    
def inicia_renderizacao():
    _frames.clear()

def termina_renderizacao():
    h, w, _ = _frames[0]['img'].shape
    plt.figure(dpi=72, figsize=(w/72, h/72))
    plt.axis('off')

    patch = plt.imshow(_frames[0]['img'][::3, ::3])
    title = plt.text(0.5, 0.95, '', transform=plt.gca().transAxes,
                     ha='center', va='top', c='white', size=18)
    plt.tight_layout()

    def renderiza_frame(f):
        patch.set_data(f['img'])
    
    skip = 3
    anim = FuncAnimation(fig=plt.gcf(),
                         func=renderiza_frame,
                         frames=tqdm(_frames[::skip], desc='Renderizando'),
                         interval=skip*1000/60)
    
    display_html(anim.to_jshtml(), raw=True)
    plt.clf()

Criando o agente:

In [7]:
from stable_baselines3 import PPO

model = PPO("MlpPolicy", env, seed=1, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


Vamos criar uma função run_episode para automatizar o processo:

In [8]:
import itertools

# A função recebe o ambiente e o agente como parâmetros
def run_episode(env, model):
    # Primeiro, inicializamos o ambiente e guardamos a observação inicial em 'obs'
    obs = env.reset()
    inicia_renderizacao()
    
    # Loop do episódio
    for t in itertools.count():
        # Nosso modelo prediz a ação 'action' a ser tomada com base na nossa observação 'obs'
        action, _states = model.predict(obs)
        
        # Tomamos a ação 'action', e recebemos uma nova observação 'obs', uma recompensa 'reward'
        # e se o episódio terminou 'done'
        obs, reward, done, info = env.step(action)
        
        # Renderiza o ambiente
        salva_frame(env)
            
        # Finaliza o episódio, caso tenha terminado
        if done:
            break
    
    # Quando terminado, fechamos o ambiente
    env.close()
    
    # Imprimindo a duração do ambiente
    print("Duração do Episódio: " + str(t+1))

    termina_renderizacao()

Vamos rodar o agente:

In [9]:
import pyvirtualdisplay

# Cria um display dentro do Colab
_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_ = _display.start()

In [10]:
run_episode(env, model)

Duração do Episódio: 200


Renderizando:   0%|          | 0/67 [00:00<?, ?it/s]

Once 
 
 Loop 
 
 Reflect

<Figure size 500x500 with 0 Axes>

Vemos acima que o agente falhou miseravelmente, ainda não havia sido treinado

**Avaliando o agente**

In [11]:
from stable_baselines3.common.evaluation import evaluate_policy

# Ambiente separado para avaliação
eval_env = gym.make("Pendulum-v1")

# Avaliando o agente
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=25, deterministic=True)

print(f"Recompensa Média: {mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Recompensa Média: -1205.22 +/- 342.3760248782656


**Treinando o agente**

In [12]:
model.learn(total_timesteps=5000000)

A saída de streaming foi truncada nas últimas 5000 linhas.
|    loss                 | 9.32       |
|    n_updates            | 22140      |
|    policy_gradient_loss | -0.00275   |
|    std                  | 0.178      |
|    value_loss           | 245        |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | -299        |
| time/                   |             |
|    fps                  | 790         |
|    iterations           | 2216        |
|    time_elapsed         | 5739        |
|    total_timesteps      | 4538368     |
| train/                  |             |
|    approx_kl            | 0.011237953 |
|    clip_fraction        | 0.0889      |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.306       |
|    explained_variance   | 0.975       |
|    learning_rate        | 0.0003      |
|    loss              

Agora com o agente treinado, vamos testá-lo novamente

In [13]:
run_episode(env, model)

Duração do Episódio: 200


Renderizando:   0%|          | 0/67 [00:00<?, ?it/s]

Once 
 
 Loop 
 
 Reflect

<Figure size 500x500 with 0 Axes>

In [14]:
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=25, deterministic=True)

print(f"Recompensa Média: {mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Recompensa Média: -142.76 +/- 98.93116729668334


Como podemos ver acima, depois de treinado, o agente consegue manter o pêndulo equilibrado! 